# Self-Attention with Relative Position Representations 구현 보충 설명

### 이름 : 전은영
 
### 코드 출처 : [tensor2tensor](https://github.com/tensorflow/tensor2tensor/tree/9e0a894034d8090892c238df1bd9bd3180c2b9a3/tensor2tensor)

## 구현할 것

- Relative Position Representation(embeddings) : RPR이라 칭하겠다
    - relation key, relation value 두 개 존재

- e 행렬 효율적으로 구하기 (행렬 곱 이용)

이 문서에서 설명은 relation key 부분만 하겠다

# 필요한 패키지 import :

In [1]:
import numpy as np

# Hyperparameter 설정 :

- b : batch size
- h : head 개수
- n : 한 sequence의 길이 (단어 개수)
- dz : RPR 벡터의 길이
- k : RPR vocab의 size 지정 (RPR vocab size : 2k+1)

In [2]:
b=2
h=3
n=4
dz=5
k=2

# RPR 구현하기

- look-up table : 상대적 위치 임베딩을 RPR vocab에서 몇 번째 임베딩 벡터를 이용할 것인 지(인덱스) 지정해준다.

- embedding vocab : 사용할 임베딩의 set. shape는 (vocab_size,dz)
  - 논문에서 $w^K,w^V$

  ![ak](https://user-images.githubusercontent.com/33515697/59223921-161ae880-8c08-11e9-986d-99fe47e4e85a.png)
  - $w^k=(w_{-k}^K,...,w_{k}^K)$ : 실제 구현 시 $w^k=(w_{0}^K,...,w_{2k}^K)$

<br>

예.

relation key look-up table = $\begin{bmatrix}
2 & 3 & 4 & 4\\ 
1 & 2 & 3 & 4\\ 
0 & 1 & 2 & 3\\ 
0 & 0 & 1 & 2
\end{bmatrix}$ 

$a_{11}^K=w^K_2$ 

: $a_{11}^K$ 임베딩으로 embedding vocab에서 인덱스가 2인 임베딩을 사용하겠다

# Look-up table 구현 :

### position_matrix(n,k)

인자
- n : 한 sequence의 길이
- k : RPR vocab의 size 지정 (RPR vocab size : 2k+1)


$a_{ij}^K$가 $w_{ind}^K$ 임베딩을 가지는데 여기서 $ind$에 해당하는 부분을 테이블로 만든다

$ind=clip(j-i,k)=max(-k,min(j-i,k))$

논문에서는 \[-k,k\]지만 인덱스로 사용할 것이므로 k를 더해서 \[0,2k\]로 범위를 바꿔준다

즉, $ind=clip(j-i,k)+k$

In [3]:
def position_matrix(n,k):
    
    range_vec=[i for i in range(n)]
    print("range_vec:",range_vec)
    
    range_mat=np.reshape(np.tile(range_vec,[n]),[n,n])
    print("range_mat (열의 index, 논문에서 j):\n",range_mat)
    print("range_mat_transpose (행의 index, 논문에서 i):\n",np.transpose(range_mat))
    
    distance_mat=range_mat-np.transpose(range_mat)
    print("distance_mat (각 원소는 j-i):\n",distance_mat)
    
    distance_mat_clipped=distance_mat
    for i in range(n):
        for j in range(n):
            if distance_mat[i][j]>k:
                distance_mat_clipped[i][j]=k
            elif distance_mat[i][j]<-k:
                distance_mat_clipped[i][j]=-k
                
    print("clipped_mat:\n",distance_mat_clipped)
    
    return distance_mat_clipped+k            

position_matrix 함수 확인

In [4]:
print("최종 look-up table (범위 [0,2k(k=%d)])\n"%k,position_matrix(n,k))

range_vec: [0, 1, 2, 3]
range_mat (열의 index, 논문에서 j):
 [[0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]]
range_mat_transpose (행의 index, 논문에서 i):
 [[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]
distance_mat (각 원소는 j-i):
 [[ 0  1  2  3]
 [-1  0  1  2]
 [-2 -1  0  1]
 [-3 -2 -1  0]]
clipped_mat:
 [[ 0  1  2  2]
 [-1  0  1  2]
 [-2 -1  0  1]
 [-2 -2 -1  0]]
최종 look-up table (범위 [0,2k(k=2)])
 [[2 3 4 4]
 [1 2 3 4]
 [0 1 2 3]
 [0 0 1 2]]


# RPR 구현 :

### position_embedding(n,dz,k)

인자
- n : 한 sequence의 길이
- dz : 하나의 RPR 벡터의 길이
- k : RPR vocab의 size 지정 (RPR vocab size : 2k+1)

위 그림의 $a^K$ (shape : (n,n,dz) ) 리턴하는 함수

In [5]:
def position_embedding(n,dz,k):
    
    np.random.seed(5)
    
    # Look-up table 생성 
    mat=position_matrix(n,k)
    print("lookup table:\n",mat)
    
    # RPR vocab_size 지정
    vocab_size=k*2+1
    
    # RPR vocab (tf에서는 variable로 학습)
    embeddings_table=np.random.randint(-10, 10, (vocab_size,dz))
    
    print("RPR vocab:\n",embeddings_table)
    
    # 인덱스만 담겨진 Look-up table을 보고, 실제 임베딩 값으로 채우기
    embeddings=np.zeros((n,n,dz))
    
    for i in range(n):
        for j in range(n):
            # Look-up table에서 index 가져오기
            pos=mat[i][j]
            embeddings[i][j]=embeddings_table[pos]

    print("embeddings shape:",embeddings.shape)
    
    return embeddings

position_embedding 함수 확인

In [6]:
embeddings=position_embedding(n,dz,k)
embeddings

range_vec: [0, 1, 2, 3]
range_mat (열의 index, 논문에서 j):
 [[0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]]
range_mat_transpose (행의 index, 논문에서 i):
 [[0 0 0 0]
 [1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]]
distance_mat (각 원소는 j-i):
 [[ 0  1  2  3]
 [-1  0  1  2]
 [-2 -1  0  1]
 [-3 -2 -1  0]]
clipped_mat:
 [[ 0  1  2  2]
 [-1  0  1  2]
 [-2 -1  0  1]
 [-2 -2 -1  0]]
lookup table:
 [[2 3 4 4]
 [1 2 3 4]
 [0 1 2 3]
 [0 0 1 2]]
RPR vocab:
 [[-7  4  5 -4  6]
 [-1 -2 -6 -3  6]
 [ 6 -3  2  5  7]
 [-3  6  2  3  1]
 [-9  5  8 -1  0]]
embeddings shape: (4, 4, 5)


array([[[ 6., -3.,  2.,  5.,  7.],
        [-3.,  6.,  2.,  3.,  1.],
        [-9.,  5.,  8., -1.,  0.],
        [-9.,  5.,  8., -1.,  0.]],

       [[-1., -2., -6., -3.,  6.],
        [ 6., -3.,  2.,  5.,  7.],
        [-3.,  6.,  2.,  3.,  1.],
        [-9.,  5.,  8., -1.,  0.]],

       [[-7.,  4.,  5., -4.,  6.],
        [-1., -2., -6., -3.,  6.],
        [ 6., -3.,  2.,  5.,  7.],
        [-3.,  6.,  2.,  3.,  1.]],

       [[-7.,  4.,  5., -4.,  6.],
        [-7.,  4.,  5., -4.,  6.],
        [-1., -2., -6., -3.,  6.],
        [ 6., -3.,  2.,  5.,  7.]]])

# e 행렬 효율적으로 구하기

e = $\begin{bmatrix}
e_{11} & ... & ... & e_{1n}\\ 
\vdots  & \vdots & \vdots & \vdots \\ 
\vdots & \vdots & \vdots & \vdots \\ 
e_{n1} & ... & ... & e_{nn}
\end{bmatrix}$

$e_{ij}=\frac{(x_iW^Q)(x_jW^K+a_{ij}^K)^T}{\sqrt{d_z}}$

행렬의 곱으로 구하는 것이 효율적이다.

하지만 지금 형태로는 행렬의 곱으로 표현할 수 없다

Transpose를 먼저 적용하면 다음과 같은 식을 얻는다

$e_{ij}=\frac{x_iW^Q(x_jW^K)^T+x_iW^Q(a_{ij}^K)^T}{\sqrt{d_z}}$

- 첫 번째 항은 원래 트랜스포머의 $e_{ij}$ => 행렬의 곱으로 표현가능
- 두 번째 항은 하나의 시퀀스에서는 행렬의 곱으로 표현이 불가능하지만, multihead, batch를 사용하면 행렬의 곱으로 구할 수 있다.

이 문서에서는 $a^K$를 행렬을 이용하여 행렬의 곱으로 어텐션 구하는 부분만 다루겠다

![original eij](https://user-images.githubusercontent.com/33515697/59229122-862f6b80-8c14-11e9-870d-2781be9a5ceb.png)


![relative aware transformer](https://user-images.githubusercontent.com/33515697/59229183-b37c1980-8c14-11e9-8b27-eb4b13708ab8.png)

정의대로 두 번째 항 행렬을 구한 것과, 논문에서 제시한 방법으로 구한 두 번째 항 행렬을 비교해보겠다.

# 임의로 input 만들기 :

input의 shape는 \[batch_size,head,depth\]

In [7]:
x=np.array([i for i in range(b*h*n*dz)]).reshape((b,h,n,dz))
x

array([[[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19]],

        [[ 20,  21,  22,  23,  24],
         [ 25,  26,  27,  28,  29],
         [ 30,  31,  32,  33,  34],
         [ 35,  36,  37,  38,  39]],

        [[ 40,  41,  42,  43,  44],
         [ 45,  46,  47,  48,  49],
         [ 50,  51,  52,  53,  54],
         [ 55,  56,  57,  58,  59]]],


       [[[ 60,  61,  62,  63,  64],
         [ 65,  66,  67,  68,  69],
         [ 70,  71,  72,  73,  74],
         [ 75,  76,  77,  78,  79]],

        [[ 80,  81,  82,  83,  84],
         [ 85,  86,  87,  88,  89],
         [ 90,  91,  92,  93,  94],
         [ 95,  96,  97,  98,  99]],

        [[100, 101, 102, 103, 104],
         [105, 106, 107, 108, 109],
         [110, 111, 112, 113, 114],
         [115, 116, 117, 118, 119]]]])

# 정의대로 두 번째 항 행렬 구하기 :

In [8]:
e_by_def=np.zeros((b,h,n,n))

for i in range(b):
    for j in range(h):
        for k in range(n):
            q=x[i][j][k]
            for l in range(n):
                e_by_def[i][j][k][l]=np.dot(q,embeddings[k][l])
                
e_by_def

array([[[[  44.,   23.,   18.,   18.],
         [ -29.,  129.,   68.,   33.],
         [  66.,  -59.,  214.,  113.],
         [  86.,   86.,  -89.,  299.]],

        [[ 384.,  203.,   78.,   78.],
         [-149.,  469.,  248.,   93.],
         [ 146., -179.,  554.,  293.],
         [ 166.,  166., -209.,  639.]],

        [[ 724.,  383.,  138.,  138.],
         [-269.,  809.,  428.,  153.],
         [ 226., -299.,  894.,  473.],
         [ 246.,  246., -329.,  979.]]],


       [[[1064.,  563.,  198.,  198.],
         [-389., 1149.,  608.,  213.],
         [ 306., -419., 1234.,  653.],
         [ 326.,  326., -449., 1319.]],

        [[1404.,  743.,  258.,  258.],
         [-509., 1489.,  788.,  273.],
         [ 386., -539., 1574.,  833.],
         [ 406.,  406., -569., 1659.]],

        [[1744.,  923.,  318.,  318.],
         [-629., 1829.,  968.,  333.],
         [ 466., -659., 1914., 1013.],
         [ 486.,  486., -689., 1999.]]]])

## 논문에서 제시한 방법으로 e 행렬 구하기

- 색칠된 부분은 위치를 시각화 하기 위함입니다 (값이 같은 것 x)

![e-1](https://user-images.githubusercontent.com/33515697/59229730-2e91ff80-8c16-11e9-93a0-00c394fbf1c6.png)
![e-2](https://user-images.githubusercontent.com/33515697/59229754-3c478500-8c16-11e9-9ab1-b74d60b6bd19.png)
![e-3](https://user-images.githubusercontent.com/33515697/59229766-3fdb0c00-8c16-11e9-8622-30c2e9d4fbe0.png)


# 1. input 준비됐으므로 pass

# 2. (b,h,n,dz)를 (n,b,h,dz)로 transpose :

batch에서 같은 위치의 길이가 dz인 vector들 끼리 묶인다

(즉, i 번째 쿼리 벡터($x_iW^Q$)들 끼리 묶음)

In [9]:
x_t=np.transpose(x,[2,0,1,3])
print(x_t)
print("x_t shape:",x_t.shape)

[[[[  0   1   2   3   4]
   [ 20  21  22  23  24]
   [ 40  41  42  43  44]]

  [[ 60  61  62  63  64]
   [ 80  81  82  83  84]
   [100 101 102 103 104]]]


 [[[  5   6   7   8   9]
   [ 25  26  27  28  29]
   [ 45  46  47  48  49]]

  [[ 65  66  67  68  69]
   [ 85  86  87  88  89]
   [105 106 107 108 109]]]


 [[[ 10  11  12  13  14]
   [ 30  31  32  33  34]
   [ 50  51  52  53  54]]

  [[ 70  71  72  73  74]
   [ 90  91  92  93  94]
   [110 111 112 113 114]]]


 [[[ 15  16  17  18  19]
   [ 35  36  37  38  39]
   [ 55  56  57  58  59]]

  [[ 75  76  77  78  79]
   [ 95  96  97  98  99]
   [115 116 117 118 119]]]]
x_t shape: (4, 2, 3, 5)


# 3. (n,b,h,dz)에서 (n,b\*h,dz)로 reshape :

batch 별 head별 분리되어 있는 것을 concat한다

In [10]:
x_t_r=np.reshape(x_t,[n,b*h,dz])
print(x_t_r)
print("x_t_r shape:",x_t_r.shape)

[[[  0   1   2   3   4]
  [ 20  21  22  23  24]
  [ 40  41  42  43  44]
  [ 60  61  62  63  64]
  [ 80  81  82  83  84]
  [100 101 102 103 104]]

 [[  5   6   7   8   9]
  [ 25  26  27  28  29]
  [ 45  46  47  48  49]
  [ 65  66  67  68  69]
  [ 85  86  87  88  89]
  [105 106 107 108 109]]

 [[ 10  11  12  13  14]
  [ 30  31  32  33  34]
  [ 50  51  52  53  54]
  [ 70  71  72  73  74]
  [ 90  91  92  93  94]
  [110 111 112 113 114]]

 [[ 15  16  17  18  19]
  [ 35  36  37  38  39]
  [ 55  56  57  58  59]
  [ 75  76  77  78  79]
  [ 95  96  97  98  99]
  [115 116 117 118 119]]]
x_t_r shape: (4, 6, 5)


# 4. 3의 결과와 RPR 행렬 곱하기 :

결과는 (n,b\*h,n)

In [11]:
emb_t=np.transpose(embeddings,[0,2,1])

x_t_r_emb_t_matmul=np.matmul(x_t_r,emb_t)

print(x_t_r_emb_t_matmul)
print("x_t_r_emb_t_matmul shape:",x_t_r_emb_t_matmul.shape)

[[[  44.   23.   18.   18.]
  [ 384.  203.   78.   78.]
  [ 724.  383.  138.  138.]
  [1064.  563.  198.  198.]
  [1404.  743.  258.  258.]
  [1744.  923.  318.  318.]]

 [[ -29.  129.   68.   33.]
  [-149.  469.  248.   93.]
  [-269.  809.  428.  153.]
  [-389. 1149.  608.  213.]
  [-509. 1489.  788.  273.]
  [-629. 1829.  968.  333.]]

 [[  66.  -59.  214.  113.]
  [ 146. -179.  554.  293.]
  [ 226. -299.  894.  473.]
  [ 306. -419. 1234.  653.]
  [ 386. -539. 1574.  833.]
  [ 466. -659. 1914. 1013.]]

 [[  86.   86.  -89.  299.]
  [ 166.  166. -209.  639.]
  [ 246.  246. -329.  979.]
  [ 326.  326. -449. 1319.]
  [ 406.  406. -569. 1659.]
  [ 486.  486. -689. 1999.]]]
x_t_r_emb_t_matmul shape: (4, 6, 4)


# 5. (n,b\*h,n)을 (n,b,h,n)으로 reshape :

In [12]:
x_t_r_emb_t_matmul_r=np.reshape(x_t_r_emb_t_matmul,[n,b,h,n])
print(x_t_r_emb_t_matmul_r)
print("x_t_r_emb_t_matmul :",x_t_r_emb_t_matmul_r.shape)

[[[[  44.   23.   18.   18.]
   [ 384.  203.   78.   78.]
   [ 724.  383.  138.  138.]]

  [[1064.  563.  198.  198.]
   [1404.  743.  258.  258.]
   [1744.  923.  318.  318.]]]


 [[[ -29.  129.   68.   33.]
   [-149.  469.  248.   93.]
   [-269.  809.  428.  153.]]

  [[-389. 1149.  608.  213.]
   [-509. 1489.  788.  273.]
   [-629. 1829.  968.  333.]]]


 [[[  66.  -59.  214.  113.]
   [ 146. -179.  554.  293.]
   [ 226. -299.  894.  473.]]

  [[ 306. -419. 1234.  653.]
   [ 386. -539. 1574.  833.]
   [ 466. -659. 1914. 1013.]]]


 [[[  86.   86.  -89.  299.]
   [ 166.  166. -209.  639.]
   [ 246.  246. -329.  979.]]

  [[ 326.  326. -449. 1319.]
   [ 406.  406. -569. 1659.]
   [ 486.  486. -689. 1999.]]]]
x_t_r_emb_t_matmul : (4, 2, 3, 4)


# 6. (n,b,h,n)을 (b,h,n,n)으로 transpose :

In [13]:
x_t_r_emb_t_matmul_r_t=np.transpose(x_t_r_emb_t_matmul_r,[1,2,0,3])
print(x_t_r_emb_t_matmul_r_t)
print("x_t_r_emb_t_matmul_r_t shape:",x_t_r_emb_t_matmul_r_t.shape)

[[[[  44.   23.   18.   18.]
   [ -29.  129.   68.   33.]
   [  66.  -59.  214.  113.]
   [  86.   86.  -89.  299.]]

  [[ 384.  203.   78.   78.]
   [-149.  469.  248.   93.]
   [ 146. -179.  554.  293.]
   [ 166.  166. -209.  639.]]

  [[ 724.  383.  138.  138.]
   [-269.  809.  428.  153.]
   [ 226. -299.  894.  473.]
   [ 246.  246. -329.  979.]]]


 [[[1064.  563.  198.  198.]
   [-389. 1149.  608.  213.]
   [ 306. -419. 1234.  653.]
   [ 326.  326. -449. 1319.]]

  [[1404.  743.  258.  258.]
   [-509. 1489.  788.  273.]
   [ 386. -539. 1574.  833.]
   [ 406.  406. -569. 1659.]]

  [[1744.  923.  318.  318.]
   [-629. 1829.  968.  333.]
   [ 466. -659. 1914. 1013.]
   [ 486.  486. -689. 1999.]]]]
x_t_r_emb_t_matmul_r_t shape: (2, 3, 4, 4)


# 결과 비교하기 :

정의로 구한 값과 행렬 곱으로 구한 값이 같다는 것을 알 수 있다.

행렬의 곱의 경우 최적화된 코드로 성능이 더 좋다

In [14]:
e_by_def,x_t_r_emb_t_matmul_r_t

(array([[[[  44.,   23.,   18.,   18.],
          [ -29.,  129.,   68.,   33.],
          [  66.,  -59.,  214.,  113.],
          [  86.,   86.,  -89.,  299.]],
 
         [[ 384.,  203.,   78.,   78.],
          [-149.,  469.,  248.,   93.],
          [ 146., -179.,  554.,  293.],
          [ 166.,  166., -209.,  639.]],
 
         [[ 724.,  383.,  138.,  138.],
          [-269.,  809.,  428.,  153.],
          [ 226., -299.,  894.,  473.],
          [ 246.,  246., -329.,  979.]]],
 
 
        [[[1064.,  563.,  198.,  198.],
          [-389., 1149.,  608.,  213.],
          [ 306., -419., 1234.,  653.],
          [ 326.,  326., -449., 1319.]],
 
         [[1404.,  743.,  258.,  258.],
          [-509., 1489.,  788.,  273.],
          [ 386., -539., 1574.,  833.],
          [ 406.,  406., -569., 1659.]],
 
         [[1744.,  923.,  318.,  318.],
          [-629., 1829.,  968.,  333.],
          [ 466., -659., 1914., 1013.],
          [ 486.,  486., -689., 1999.]]]]),
 array([[[[  44., 